In [6]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
tf.enable_eager_execution ()

# Neural Ordinary Differential Equations

## Two different ways to model things

When considering a problem of process modeling we - usually - have set of N data points pairs ${(x_n, y_n)}; 1 \le n \le N$. This dataset comes from observing some process out in the wild. What we would like to do is to build mapping $f$ that will approximate the very process we observed. 

We will compare two approaches - quite popular **regression** and the topic of this presentation **ordinary differential equations**.

### Regression

We can write regression in general from as:
$$Y = f(X, \beta) + \epsilon$$

Of course, $f$ can take various form - from simple matrix multiplication to extremally complex like complicated neural nets. What is most important is the fact that we describe the relationship _directly_. 


### Ordinary Differential Equations

On the other hand, when using the ODE approach instead of using a _direct_ approach, we can observe _rate of change_ - which in mathematical terms means we are toying with derivatives. We can describe this relationship as:
$$\frac{dy}{dx} = f'(x),$$
where f is a function we are trying to approximate. To calculate $f$, we need to integrate it.

The general idea is that instead of approximating the function itself, we can approximate it's derivative.


## ODE Refresher

### What are ODEs?
Straight from Wikipedia:
> In mathematics, an ordinary differential equation (ODE) is a differential equation containing one or more functions of one independent variable and the derivatives of those functions. The term ordinary is used in contrast with the term partial differential equation which may be with respect to more than one independent variable.

Let $y = f(x)$ then equation of the form:
$$F(x, y, y', ..., y^{(n-1)})=y^{(n)}$$
is called explicit ordinary differential equation of order n.

### Initial value problem in ODEs

One of the most common problems is that of _initial value problem_. When modeling system this usually means that we want to know how a system will evolve in "time" given some initial conditions.

#### Sample Problem

$$\frac{dy(t)}{dt} = 10 - t, y(0) = - 1$$

###### General Solution

$$\frac{dy(t)}{dt} = 10 - t$$

$$dy(t) = (10 - t)dt$$

$$\int dy(t)dt = \int (10 -t)dt$$

$$y = 10t - \frac{t^2}{2} + C$$

###### Specific solution

$$-1 = 10(0) - \frac{0}{2} + C$$

$$C = -1$$

Solution: $y = 10t - \frac{t^2}{2} - 1$

## ResNet and Euler's Method

### What is ResNet?

ResNet is a network architecture introduced in 2015, taking _deep_ in deep learning to a whole new level. The motivation behind creating this network is highly empirical - creators (not only them) observed issue the named _degradation problem_. The observed that adding layers not only does not improve the overall accuracy of the network but worsens it. That implies that added layers are not able to learn even identity function - which would be enough to keep accuracy on the same level.

Let's set that ordered set of layers should should learn some function $g$. If our net is able to learn how to approximate this function then it should be able to learn how to approximate residual function $f(x) = g(x) - x$. From that we have $g(x) = f(x) + x$

### Residual Block

Residual block can be considered as a mini neural net, where _input of this net is also added to the output of the net_. We may think about it as:
$$fr^l(x) = f(x) + Wx$$
Where $l$ represents a number of layers in a mini neural net. $W$, on the other hand, is a matrix that allows simple linear transformation, for example, to make "dimensions compile". Usually, it's just an identity matrix.

#### ResNet as Euler Discretization

Now... let's get back to ODEs. One of the simplest approaches to numerically solve ODE is _Euler's Discretization_. Idea behind this method goes as follows:
1. Consider $y' = f(x, y)$ given initial condition $y_0 = y(x_0)$
2. Let's suppose our step size is of size $t$. We will use this value to update value of $x$: $x_{n+1} = x_{n} + t$.
3. From definition of derivative we have $y' = \frac{\Delta y}{t}$, simple manipulation gives us $\Delta y = tf(x_n, y_n)$
4. From everything above we get $y_{n+1} = y_n + \Delta y$ and $y_{n + 1} = y_n + tf(x_n, y_n)$.

It turns out we can eaisly represent ResNet block in similar form, particularly like this:
$$h(t+1) = h(t) + f(h(t))$$
which is extremally similar to:
$$y_{n+1} = y_n + \Delta y$$

Hmm... 



#### Making Discretization continous

Authors of Neural ODE asked a question (not first though) and provided implementable answers to the following question: _what will happen if we make as small steps as possible?_ Making our discretization - continuous. They described the state of the neural network, or rather change of the state in the following form:

$$\frac{dh(t)}{dt} = f(t, h(t); \theta)$$

Where t is _depth_ of our network, which could be imagined as a continuous number of layer. So $h(t_0)$ would be input to the network and $h(t_1)$ could be the output of the network. 

##### Forward pass 

For starter let's think how to do forward pass in a network defined as above. The most straight forward approach seems to make sense:

$$h(t_1) = h(t_0) + \int_{t_0}^{t_1}f(t, h(t); \theta)$$

That looks very nice, we can easily write formula for loss function:

$$ L(h(t_1), y_{expected}) = L(h(t_0) + \int_{t_0}^{t_1}f(t, h(t); \theta), y_{expected}) = L(ODESolve(h(t_0), f, t_0, t_1, \theta)) $$

$ODESolve$ is a _black box ode solver _ from our perspective.

## Backpropagation

To fully formulate our network we need a way to somehow update the network's parameters so that our network can learn. 

### Naive Approach

The most naive approach would be just to backpropagate due to: number of steps required to solve an equation, needing to store all the activations, numerical errors, lack of guarantees on differentiability. 

### What is the adjoint method?

Take from: https://rkevingibson.github.io/blog/neural-networks-as-ordinary-differential-equations/, as this is a simple and neat explanation.

Suppose we have 2 known matrices $A$ and $C$, and a known vector u. We would like to compute the product:

$$u^TB \text{ such that } AB = C$$

A most natural approach would be to solve a linear system to find an unknown matrix $B$, then compute the product. Let's use mathigical trick to solve this problem. Let's find a vector $v$ and compute:

$$ v^TC \text{ such that } A^Tv = u $$

We can reinterpet our initial problem as:

$$v^TC = v^TAB = (A^Tv)^TB = u^TB$$

Using this transformation, we have reduced the problem from solving for a matrix to solving for a vector.

### And what all of this has to do with Neural ODEs?

For technical details please see the original Neural ODE paper, especially Appendix B, here we will try to explain the intuition behind it. Based on: https://jontysinai.github.io/jekyll/update/2019/01/18/understanding-neural-odes.html.

To optimize loss, we need gradients w.r.t. all free parameters (usually represented by $\theta$), because those are parameters that will be updated.

The first step (similarly to "standard" backprop) is to compute the gradient of the loss with respect to the hidden states:

$$\dfrac{\delta L}{\delta h(t)}$$

The hidden state itself is dependent on time/depth, so we also can take derivative w.r.t. to time. Consider that this time derivative will be a _reverse traversal of the hidden states_. This may be numerically intensive - and this is when the adjoint method comes into play.

In order to do so we will define _adjoint state vector_, that will keep track of the time dynamics:

$$a(t) = -\dfrac{\delta L}{\delta h(t)}$$

The authors prove that the adjoint state defined like this is can be passed to _black box ode solver_, to calculate gradients w.r.t $t_0$.

$$a(t_0) = \int_{t_1}^{t_0} -a(t)^T \dfrac{\delta f(t, h(t); \theta_t)}{\delta h(t)}dt$$.

Based on this we are able to define all required gradients, that is:

$$\frac{d L}{d \theta} =-\int_{t_1}^{t_0} a(t)^T \dfrac{\delta f(t, h(t); \theta_t)}{\delta \theta}dt $$

$$\frac{dL}{dt_1} = a(t_1)f(a(t_1), t_1, \theta)$$

$$\frac{dL}{dt_0} = a(t_1) -\int_{t_1}^{t_0} a(t)^T \dfrac{\delta f(t, h(t); \theta_t)}{\delta t}dt $$

To sum it beautifully:

> _Backpropagation using the adjoint method: the adjoint state is a vector with its own time-dependent dynamics, only the trajectory runs backward in time. We can solve for the gradients using an ODE solver for the adjoint time derivative, starting at $t_1$._ [source](https://jontysinai.github.io/jekyll/update/2019/01/18/understanding-neural-odes.html)

### Combining it all together!

[source](https://jontysinai.github.io/jekyll/update/2019/01/18/understanding-neural-odes.html)

How would we solve a problem using this approach?

1. Instead of modelling this relationship direcatly, we model derivative:

$$\frac{dy}{dx} = f(x, y)$$

2. We parametrise approximation by a neural network with hidden states $h(t)$, with $h(t_0) = x$, $h(t_1) = y$. Function approximation problem now has form of (f is a neural network):

$$\frac{dh(t)}{dt} = f(t, h(t), \theta)$$


3. Based on the above we can numerically integrate state, so we get network response:

$$\hat{y} = h(t_1) = \int_{t_0}^{t_1} f(t, h(t), \theta)dt$$.

4. Free parameters of our problem are $t_0, t_1, \theta$. We optimize free parameters with the help of the adjoint method.

### Wait - but why?

- Adaptive computation
- Parameter efficency

In [11]:
class NeuralODE:
    def __init__(self, model: tf.keras.Model, t=[0,1]):
        self._t = t
        self._model = model
        self.__output_dim_product = None
        self.__output_gradients_dim_product = None
        self.__batch_size = None
        self.__number_of_parameters = None
        
    
    def forward_pass(self, inputs, rtol=1e-06, atol=1e-12):
        out = tf.contrib.integrate.odeint(
            func = lambda _y, _t: self._model(inputs=(_t, _y)),
            y0=inputs,
            t=tf.to_float(self._t),
            rtol=rtol,
            atol=atol,
            full_output=True
        )
        return out
    
    def backward_pass(self, outputs, output_gradients, rtol=1e-06, atol=1e-12):
        # Ugly code, should be improved
        self.__output_dim_product = self.__number_of_params(outputs)
        self.__output_gradients_dim_product = self.__number_of_params(output_gradients)
        self.__batch_size = self.__raw_shape(outputs)[0]
        flat_state_tensor = self.__flat_state_tensor(output_gradients,outputs)
        self.__number_of_parameters = self.__number_of_params(flat_state_tensor) - self.__output_gradients_dim_product - self.__output_dim_product
        # TO IMPLEMENT PART
        out, info_dict = tf.contrib.integrate.odeint(
            func = lambda _y, _t: -1 * self._backward_dynamics(_t,  _y),
            y0=flat_state_tensor,
            t=tf.cast([0, 1], tf.float32),
            rtol=rtol,
            atol=atol,
            full_output=True
        )

        layers_dims = [self.__raw_shape(weight) for weight in self._model.weights]
        layers_size = [self.__number_of_params(weight) for weight in self._model.weights]
        adjoint, hidden, gradients = tf.split(
            out[-1], 
            [self.__output_gradients_dim_product , self.__output_dim_product, np.sum(layers_size)]
        )
        layer_params_dims = zip(tf.split(gradients, layers_size), layers_dims)
        reshaped = [tf.reshape(param_dim[0], param_dim[1]) for param_dim in layer_params_dims]
        return [
            tf.reshape(hidden, [self.__batch_size, -1]), *reshaped
        ]
    # TO IMPLEMENT PART
    def _backward_dynamics(self, t, flat_state_tensor):
        adjoint, hidden, layers = tf.split(
            flat_state_tensor, 
            [self.__output_gradients_dim_product, self.__output_dim_product, self.__number_of_parameters]
        )
        # WHAT ARE THOSE?
        at = -tf.reshape(adjoint, [self.__batch_size, -1])
        ht = tf.reshape(hidden, [self.__batch_size, -1])
        with tf.GradientTape() as g:
            g.watch(ht)
            ht_new = self._model(inputs=(t, ht))
        # WHY THIS GRADIENT?
        gradients = g.gradient(
            target=ht_new, 
            sources= [ht] + self._model.weights,
            output_gradients=at
        )
        flat_layers = [tf.reshape(weight, [-1]) for weight in gradients]
        resulting_state = tf.concat([adjoint, *flat_layers], axis=0)
        return resulting_state
    
    def __flat_state_tensor(self, output_gradients, outputs):
        grad_weights = [tf.zeros_like(w) for w in self._model.weights]
        flat_layers = [tf.reshape(weight, [-1]) for weight in grad_weights]
        state = (tf.reshape(output_gradients, [-1]), tf.reshape(outputs, [-1]), *flat_layers)
        flat_state_tensor = tf.concat(state, 0)
        return flat_state_tensor
    
        
    def __number_of_params(self, tensor):
        raw_dims = self.__raw_shape(tensor)
        return np.prod(raw_dims)
    
    def __raw_shape(self, tensor):
        s = tensor.get_shape()
        raw_dims = tuple([s[i].value for i in range(0, len(s))])
        return raw_dims
        

## Sample problem - MNIST

In [12]:
# Parameters
learning_rate = 0.0001
num_steps = 100
batch_size = 128
display_step = 300

# Network Parameters
node_dim = 256 # 1st layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)


# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

in_layer = tf.keras.layers.Dense(node_dim)
out_layer = tf.keras.layers.Dense(num_classes)

class Lambda(tf.keras.Model):
    def __init__(self):
        super(Lambda, self).__init__(name="Module")
        self.dense_1 = tf.keras.layers.Dense(node_dim)
        self.dense_2 = tf.keras.layers.Dense(node_dim)
        
    def call(self, inputs, **kwargs):
        t, x = inputs
        h = self.dense_1(x)
        return  self.dense_2(h)

neural_ode = NeuralODE(Lambda())
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
loss_history = []
for step in range(num_steps):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    in_result = in_layer(batch_x)
    node_output, info_dict = neural_ode.forward_pass(in_result)
    last_step = node_output[-1]
    prediction = out_layer(last_step)
    with tf.GradientTape() as g: 
        g.watch(prediction)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=batch_y))
    loss_history.append(loss.numpy())
    dLossdLast = g.gradient(loss, prediction)
    dWeightsdOut = tf.matmul(tf.transpose(last_step), dLossdLast)
    
    dLoss = tf.matmul(dLossdLast, tf.transpose(out_layer.weights[0]))
    hidden, *dWeightsdODE = neural_ode.backward_pass(last_step, dLoss) 
    
    back_hidden = tf.matmul(hidden, tf.transpose(neural_ode._model.dense_2.weights[0]))
    back_hidden = tf.matmul(back_hidden, neural_ode._model.dense_1.weights[0])
    step_back = tf.matmul(back_hidden, tf.transpose(in_layer.weights[0]))
    dWeightsdIn = tf.matmul(tf.transpose(step_back), in_result)

    optimizer.apply_gradients(zip(dWeightsdODE, neural_ode._model.trainable_weights))
    optimizer.apply_gradients(zip([dWeightsdOut, tf.reduce_mean(dLossdLast, axis=0)], out_layer.trainable_weights))
    optimizer.apply_gradients(zip([dWeightsdIn, tf.reduce_mean(hidden, axis=0)], in_layer.trainable_weights))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
{'num_func_evals': <tf.Tensor: id=292552, shape=(), dtype=int32, numpy=43>, 'integrate_points': <tf.Tensor: id=292547, shape=(7,), dtype=float32, numpy=
array([1.        , 0.2232766 , 0.18914576, 0.37209553, 0.5840182 ,
       0.8200452 , 1.0748899 ], dtype=float32)>, 'error_ratio': <tf.Tensor: id=292553, shape=(7,), dtype=float32, numpy=
array([1.0641342e+03, 1.3534608e+00, 6.9748753e-01, 2.8312925e-01,
       3.4458068e-01, 4.0238574e-01, 4.3884486e-01], dtype=float32)>}
{'num_func_evals': <tf.Tensor: id=298088, shape=(), dtype=int32, numpy=43>, 'integrate_points': <tf.Tensor: id=298083, shape=(7,), dtype=float32, numpy=
array([1.        , 0.22328267, 0.18882613, 0.37140164, 0.5819026 ,
       0.81689525, 1.0714382 ], dtype=float32)>, 'error_ratio': <tf.Tensor: id=298089, shape=(7,), dtype=float32

{'num_func_evals': <tf.Tensor: id=384341, shape=(), dtype=int32, numpy=43>, 'integrate_points': <tf.Tensor: id=384336, shape=(7,), dtype=float32, numpy=
array([1.        , 0.22332443, 0.18883575, 0.37134784, 0.58152217,
       0.8162141 , 1.0707641 ], dtype=float32)>, 'error_ratio': <tf.Tensor: id=384342, shape=(7,), dtype=float32, numpy=
array([1.0629950e+03, 1.3660688e+00, 7.0012432e-01, 2.9159003e-01,
       3.4011075e-01, 3.9340273e-01, 4.3982181e-01], dtype=float32)>}
{'num_func_evals': <tf.Tensor: id=389719, shape=(), dtype=int32, numpy=43>, 'integrate_points': <tf.Tensor: id=389714, shape=(7,), dtype=float32, numpy=
array([1.        , 0.22330394, 0.1889291 , 0.3713855 , 0.5798601 ,
       0.8136233 , 1.0674163 ], dtype=float32)>, 'error_ratio': <tf.Tensor: id=389720, shape=(7,), dtype=float32, numpy=
array([1.0634833e+03, 1.3620726e+00, 7.0292854e-01, 3.0320898e-01,
       3.3311722e-01, 3.9146703e-01, 4.3356884e-01], dtype=float32)>}
{'num_func_evals': <tf.Tensor: id=395097, sh

KeyboardInterrupt: 

In [ ]:
# GRAPH ERRORS
print(loss_history)
plt.plot(loss_history)


### More advanced problems

- Scalable and intertivle normalizing flows
- Continous time-sries models

Sources:
 - https://github.com/kmkolasinski/deep-learning-notes/tree/master/seminars/2019-03-Neural-Ordinary-Differential-Equations
 - https://rkevingibson.github.io/blog/neural-networks-as-ordinary-differential-equations/
 - https://jontysinai.github.io/jekyll/update/2019/01/18/understanding-neural-odes.html
 - https://arxiv.org/abs/1806.07366
 - https://pl.wikipedia.org/wiki/Regresja_(statystyka)
 - https://en.wikipedia.org/wiki/Ordinary_differential_equation
 - https://en.wikipedia.org/wiki/Initial_value_problem
 - http://tutorial.math.lamar.edu/Classes/DE/Modeling.aspx
 - https://en.wikipedia.org/wiki/Sensitivity_analysis
 - https://wizardforcel.gitbooks.io/tensorflow-101-sjchoi86/mlp_mnist_simple.html
 - Reprezentacja stanu sieci neuronowych przy pomocy zwyczajnych równań różniczkowych - Piotr Lewandowski